In [ ]:
# MAPPING IS-8 SECTORS ACROSS THE UK
# This is to support ECO article on comparing IS-8 sector performance internationally
# We want to show where firms operate/employment is concentrated for the growth sectors
# Will Shepherd, November 2025

import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import os
import eco_style 
alt.themes.enable("light")
import requests
import json
import time
from tqdm import tqdm
import io
from shapely.geometry import Point

# The IS-8 sectors
    - Advanced Manufacturing
    - Clean Energy Industries
    - Creative Industries
    - Defence
    - Digital and Technologies
    - Financial Services
    - Life sciences
    - Professional and Business Services

We want to manually assemble an SIC lookup to identify these sectors - based on:
https://www.gov.uk/government/publications/industrial-strategy/industrial-strategy-sector-definitions-list

In [168]:
# DOWNLOAD BRES EMPLOYMENT BY 5DIG SIC AND LAD FROM NOMIS API

# Create the output directory if it doesn't exist
output_dir = "nomis_dump"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# This is the base URL for BRES data
base_url = "https://www.nomisweb.co.uk/api/v01/dataset/NM_189_1.data.csv"

# This is list of LAD geographies - copy this from your link from NOMIS API
GEO_LIST_STR = "1778384897...1778384901,1778384941,1778384950,1778385143...1778385146,1778385159,1778384902...1778384905,1778384942,1778384943,1778384956,1778384957,1778385033...1778385044,1778385124...1778385138,1778384906...1778384910,1778384958,1778385139...1778385142,1778385154...1778385158,1778384911...1778384914,1778384954,1778384955,1778384965...1778384972,1778385045...1778385058,1778385066...1778385072,1778384915...1778384917,1778384944,1778385078...1778385085,1778385100...1778385104,1778385112...1778385117,1778385147...1778385153,1778384925...1778384928,1778384948,1778384949,1778384960...1778384964,1778384986...1778384997,1778385015...1778385020,1778385059...1778385065,1778385086...1778385088,1778385118...1778385123,1778385160...1778385192,1778384929...1778384940,1778384953,1778384981...1778384985,1778385004...1778385014,1778385021...1778385032,1778385073...1778385077,1778385089...1778385099,1778385105...1778385111,1778384918...1778384924,1778384945...1778384947,1778384951,1778384952,1778384973...1778384980,1778384998...1778385003,1778384959,1778385193...1778385246"

INDUSTRY_CODES = "134218728,134218838,134218848,134218858,134218868,134218878,134218888,134218918,134218938,134218948,134218958,134218968,134218978,134218988,134218998,134219008,134219018,134219028,134219138,134219148,134219158,134219168,134219178,134219188,134219198,134219218,134219228,134219338,134219349,134219357,134219358,134219368,134219428,134219828,134219928,134220028,134220128,134220838,134220848,134220938,134220948,134222829,134222830,134222928,134223828,134223928,134224828,134224938,134225018,134225838,134225848,134226638,134226648,134226658,134226718,134226828,134227628,134227838,134227848,134227858,134227928,134228038,134228048,134228118,134228138,134228148,134228239,134228240,134228247,134228248,134228339,134228340,134228348,134228438,134228448,134228458,134228538,134228549,134228550,134228559,134228560,134228568,134228578,134228588,134228618,134228638,134228648,134228738,134228748,134228758,134228768,134228778,134228788,134228798,134229728,134230828,134230928,134231028,134231638,134231649...134231651,134231659,134231667,134231668,134231678,134231688,134231718,134231838,134231848,134231859,134231860,134231869,134231870,134231918,134231928,134232038,134232118,134232838,134232848,134232928,134233828,134233938,134233948,134233958,134233968,134234018,134234838,134234848,134234939,134234947,134234948,134234958,134234968,134235018,134235838,134235849,134235857,134235858,134235868,134235929...134235931,134236828,134236929,134236937,134237838,134237848,134237858,134237868,134237878,134237888,134237898,134237928,134238029,134238030,134238139,134238140,134238148,134238238,134238248,134238258,134238318,134238328,134238828,134238928,134239838,134239918,134239938,134239948,134239958,134240018,134240838,134240848,134240858,134240868,134240918,134240928,134241038,134241048,134241138,134241148,134241158,134241168,134241218,134241238,134241248,134241338,134241348,134241358,134241368,134241378,134241418,134241428,134241638,134241718,134241828,134241928,134242038,134242048,134242058,134242068,134242138,134242148,134242158,134242168,134242178,134242188,134242238,134242248,134242258,134242268,134242838,134242848,134242938,134243018,134243028,134243128,134243228,134243338,134243348,134243438,134243448,134243458,134243638,134243648,134243658,134243668,134243718,134243838,134243848,134243928,134244029,134244037,134244128,134244239...134244242,134244248,134244328,134244429,134244430,134244528,134244838,134244848,134244928,134245038,134245048,134245058,134245128,134245238,134245248,134245628,134245838,134245848,134245859,134245860,134245868,134245878,134245938,134245948,134245958,134245968,134245978,134246018,134246029,134246030,134246138,134246218,134246638,134246649...134246651,134246658,134246668,134246678,134246688,134246718,134246828,134246929...134246931,134247038,134247048,134247838,134247848,134247928,134248028,134248128,134248638,134248648,134248718,134248738,134248748,134248758,134248818,134249838,134249848,134249858,134249928,134250028,134250129,134250137,134250228,134250638,134250718,134250838,134250848,134250858,134250868,134250878,134250888,134250898,134250918,134250928,134252838,134252848,134252858,134252868,134252938,134252948,134252958,134253028,134253728,134254728,134255838,134255848,134255938,134255948,134256038,134256048,134256728,134258828,134258929,134258930,134259838,134259848,134259858,134259938,134259948,134260638,134260718,134260838,134260848,134260858,134260938,134260948,134261018,134261038,134261048,134261058,134261069,134261070,134261118,134261638,134261719,134261727,134262839,134262840,134262918,134262928,134263038,134263048,134263128,134263838,134263848,134263858,134263868,134263878,134263888,134263898,134263908,134263918,134263938,134263948,134263958,134263968,134264038,134264048,134264058,134264069,134264070,134264078,134264088,134264098,134264108,134264118,134264138,134264148,134264159,134264167,134264168,134264178,134264188,134264198,134264208,134264219,134264227,134264238,134264248,134264338,134264348,134264358,134264368,134264378,134264388,134264418,134264439,134264447,134264448,134264458,134264468,134264478,134264488,134264498,134264628,134264838,134264918,134264938,134264948,134264958,134264968,134264978,134264988,134265018,134265028,134265138,134265149,134265157,134265158,134265238,134265248,134265258,134265268,134265319,134265327,134265338,134265348,134265358,134265368,134265378,134265438,134265449,134265450,134265458,134265469,134265477,134265478,134265488,134265498,134265509,134265510,134265517,134265519,134265527,134265538,134265548,134265618,134265638,134265718,134266828,134266928,134267039,134267047,134267048,134267118,134267138,134267148,134267228,134267828,134267928,134268028,134268128,134268829,134268830,134268938,134268948,134269829...134269831,134269939...134269941,134269947,134269948,134269958,134269969...134269971,134270018,134270828,134270929,134270930,134272828,134272929,134272930,134272937,134273028,134273628,134273829...134273831,134273938,134274018,134274029,134274030,134275838,134275848,134275858,134275869,134275870,134275918,134275938,134276018,134276839...134276841,134276848,134276859...134276861,134276868,134276928,134277828,134277928,134278828,134278928,134279028,134279628,134279739,134279740,134279748,134279758,134279818,134280838,134280848,134281638,134281718,134281838,134281919,134281920,134281929...134281933,134281937,134282029...134282034,134282638,134282649,134282650,134282657,134282719,134282720,134282727,134282838,134282848,134282929,134282930,134283028,134283838,134283848,134283918,134283938,134283948,134284018,134284028,134285828,134285929,134285930,134285937,134286038,134286048,134286829,134286830,134286837,134286929...134286931,134287828,134287938,134287949,134287957,134288839,134288840,134288849,134288850,134288857,134288928,134289838,134289918,134289928,134290838,134290848,134290928,134291828,134291929...134291931,134291937,134292028,134292629,134292630,134292637,134292728,134294838,134294848,134294938,134294948,134295019,134295027,134295038,134295048,134295058,134295069,134295070,134295079,134295080,134295118,134295128,134295829,134295837,134295928,134296028,134296838,134296848,134297629,134297637,134297828,134297928,134298028,134298828,134298938,134298949...134298951,134298957,134299019,134299027,134299028,134299838,134299918,134299928,134300029,134300030,134300639,134300640,134300648,134300718,134301838,134301848,134301858,134301938,134301948,134301958,134301968,134301978,134302028,134302828,134302928,134303038,134303048,134303138,134303149,134303150,134303238,134303248,134303258,134303318,134303328,134303829,134303830,134303938,134303948,134303958,134304628,134304828,134304928,134305028,134305628,134305828,134306638,134306718,134307738,134307748,134307758,134307768,134308739,134308740,134308748,134308758,134308768,134309728,134310838,134310848,134310858,134310919,134310927,134310938,134311018,134311838,134311848,134311928,134312638,134312648,134312718,134312838,134312848,134312938,134312948,134312958,134312968,134312978,134313018,134313738,134313748,134313758,134313768,134313818,134314728,134315828,134315928,134316728"

INDUSTRY_LIST = "134218728,134218838,134218848,134218858,134218868,134218878,134218888,134218918,134218938,134218948,134218958,134218968,134218978,134218988,134218998,134219008,134219018,134219028,134219138,134219148,134219158,134219168,134219178,134219188,134219198,134219218,134219228,134219338,134219349,134219357,134219358,134219368,134219428,134219828,134219928,134220028,134220128,134220838,134220848,134220938,134220948,134222829,134222830,134222928,134223828,134223928,134224828,134224938,134225018,134225838,134225848,134226638,134226648,134226658,134226718,134226828,134227628,134227838,134227848,134227858,134227928,134228038,134228048,134228118,134228138,134228148,134228239,134228240,134228247,134228248,134228339,134228340,134228348,134228438,134228448,134228458,134228538,134228549,134228550,134228559,134228560,134228568,134228578,134228588,134228618,134228638,134228648,134228738,134228748,134228758,134228768,134228778,134228788,134228798,134229728,134230828,134230928,134231028,134231638,134231649...134231651,134231659,134231667,134231668,134231678,134231688,134231718,134231838,134231848,134231859,134231860,134231869,134231870,134231918,134231928,134232038,134232118,134232838,134232848,134232928,134233828,134233938,134233948,134233958,134233968,134234018,134234838,134234848,134234939,134234947,134234948,134234958,134234968,134235018,134235838,134235849,134235857,134235858,134235868,134235929...134235931,134236828,134236929,134236937,134237838,134237848,134237858,134237868,134237878,134237888,134237898,134237928,134238029,134238030,134238139,134238140,134238148,134238238,134238248,134238258,134238318,134238328,134238828,134238928,134239838,134239918,134239938,134239948,134239958,134240018,134240838,134240848,134240858,134240868,134240918,134240928,134241038,134241048,134241138,134241148,134241158,134241168,134241218,134241238,134241248,134241338,134241348,134241358,134241368,134241378,134241418,134241428,134241638,134241718,134241828,134241928,134242038,134242048,134242058,134242068,134242138,134242148,134242158,134242168,134242178,134242188,134242238,134242248,134242258,134242268,134242838,134242848,134242938,134243018,134243028,134243128,134243228,134243338,134243348,134243438,134243448,134243458,134243638,134243648,134243658,134243668,134243718,134243838,134243848,134243928,134244029,134244037,134244128,134244239...134244242,134244248,134244328,134244429,134244430,134244528,134244838,134244848,134244928,134245038,134245048,134245058,134245128,134245238,134245248,134245628,134245838,134245848,134245859,134245860,134245868,134245878,134245938,134245948,134245958,134245968,134245978,134246018,134246029,134246030,134246138,134246218,134246638,134246649...134246651,134246658,134246668,134246678,134246688,134246718,134246828,134246929...134246931,134247038,134247048,134247838,134247848,134247928,134248028,134248128,134248638,134248648,134248718,134248738,134248748,134248758,134248818,134249838,134249848,134249858,134249928,134250028,134250129,134250137,134250228,134250638,134250718,134250838,134250848,134250858,134250868,134250878,134250888,134250898,134250918,134250928,134252838,134252848,134252858,134252868,134252938,134252948,134252958,134253028,134253728,134254728,134255838,134255848,134255938,134255948,134256038,134256048,134256728,134258828,134258929,134258930,134259838,134259848,134259858,134259938,134259948,134260638,134260718,134260838,134260848,134260858,134260938,134260948,134261018,134261038,134261048,134261058,134261069,134261070,134261118,134261638,134261719,134261727,134262839,134262840,134262918,134262928,134263038,134263048,134263128,134263838,134263848,134263858,134263868,134263878,134263888,134263898,134263908,134263918,134263938,134263948,134263958,134263968,134264038,134264048,134264058,134264069,134264070,134264078,134264088,134264098,134264108,134264118,134264138,134264148,134264159,134264167,134264168,134264178,134264188,134264198,134264208,134264219,134264227,134264238,134264248,134264338,134264348,134264358,134264368,134264378,134264388,134264418,134264439,134264447,134264448,134264458,134264468,134264478,134264488,134264498,134264628,134264838,134264918,134264938,134264948,134264958,134264968,134264978,134264988,134265018,134265028,134265138,134265149,134265157,134265158,134265238,134265248,134265258,134265268,134265319,134265327,134265338,134265348,134265358,134265368,134265378,134265438,134265449,134265450,134265458,134265469,134265477,134265478,134265488,134265498,134265509,134265510,134265517,134265519,134265527,134265538,134265548,134265618,134265638,134265718,134266828,134266928,134267039,134267047,134267048,134267118,134267138,134267148,134267228,134267828,134267928,134268028,134268128,134268829,134268830,134268938,134268948,134269829...134269831,134269939...134269941,134269947,134269948,134269958,134269969...134269971,134270018,134270828,134270929,134270930,134272828,134272929,134272930,134272937,134273028,134273628,134273829...134273831,134273938,134274018,134274029,134274030,134275838,134275848,134275858,134275869,134275870,134275918,134275938,134276018,134276839...134276841,134276848,134276859...134276861,134276868,134276928,134277828,134277928,134278828,134278928,134279028,134279628,134279739,134279740,134279748,134279758,134279818,134280838,134280848,134281638,134281718,134281838,134281919,134281920,134281929...134281933,134281937,134282029...134282034,134282638,134282649,134282650,134282657,134282719,134282720,134282727,134282838,134282848,134282929,134282930,134283028,134283838,134283848,134283918,134283938,134283948,134284018,134284028,134285828,134285929,134285930,134285937,134286038,134286048,134286829,134286830,134286837,134286929...134286931,134287828,134287938,134287949,134287957,134288839,134288840,134288849,134288850,134288857,134288928,134289838,134289918,134289928,134290838,134290848,134290928,134291828,134291929...134291931,134291937,134292028,134292629,134292630,134292637,134292728,134294838,134294848,134294938,134294948,134295019,134295027,134295038,134295048,134295058,134295069,134295070,134295079,134295080,134295118,134295128,134295829,134295837,134295928,134296028,134296838,134296848,134297629,134297637,134297828,134297928,134298028,134298828,134298938,134298949...134298951,134298957,134299019,134299027,134299028,134299838,134299918,134299928,134300029,134300030,134300639,134300640,134300648,134300718,134301838,134301848,134301858,134301938,134301948,134301958,134301968,134301978,134302028,134302828,134302928,134303038,134303048,134303138,134303149,134303150,134303238,134303248,134303258,134303318,134303328,134303829,134303830,134303938,134303948,134303958,134304628,134304828,134304928,134305028,134305628,134305828,134306638,134306718,134307738,134307748,134307758,134307768,134308739,134308740,134308748,134308758,134308768,134309728,134310838,134310848,134310858,134310919,134310927,134310938,134311018,134311838,134311848,134311928,134312638,134312648,134312718,134312838,134312848,134312938,134312948,134312958,134312968,134312978,134313018,134313738,134313748,134313758,134313768,134313818,134314728,134315828,134315928,134316728".split(',')

# Function to split a list into chunks
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

# Iterate by chunks of industry codes

# Split the industry list into manageable chunks of 20
industry_chunks = list(chunk_list(INDUSTRY_LIST, 20))
all_dataframes = []

print(f"Starting download ... Will make {len(industry_chunks)} separate API calls.")

for i, industry_chunk in enumerate(tqdm(industry_chunks, desc="Downloading Chunks")):
    
    # Define the filename for this chunk
    chunk_filename = os.path.join(output_dir, f"bres_chunk_{i}.csv")

    # Skip if we already downloaded this file
    if os.path.exists(chunk_filename):
        print(f"Chunk {i} already downloaded. Skipping...")
        continue

    #Join the shunk of industry codes back into a comma-separated string
    industry_str_chunk = ",".join(industry_chunk)

    # Define all parameters for this request
    params = {
        'geography': GEO_LIST_STR,
        'date': 'latest',
        'industry': industry_str_chunk,
        'employment_status': '4',
        'measure':'1',
        'measures': '20100'
    #    'select': 'GEOGRAPHY_CODE, GEOGRAPHY_NAME,INDUSTRY, OBS_VALUE'
    }

    try:
        response = requests.get(base_url, params=params)

        # Raise an error if the API returns 4xx or 5xx status code
        response.raise_for_status()

        # Use io.StringIO to read the text content into a dataframe
        chunk_df = pd.read_csv(io.StringIO(response.text))

        # Save the chunk to a file
        chunk_df.to_csv(chunk_filename, index=False)

    except requests.exceptions.HTTPError as err:
        print(f"\n--- HTTP ERROR on chunk {i} ---")
        print(f"Error: {err}")
        print("This chunk failed, likely because it also exceeded the cell limit.")
        print(f"Response from server: {response.text[:200]}...") # Print first 200 chars of error
    except pd.errors.ParserError as err:
        print(f"\n--- PANDAS ERROR on chunk {i} ---")
        print(f"Error: {err}")
        print("This means the server did not return a CSV. It was probably an error page.")
        print(f"Response from server: {response.text[:200]}...")
    except Exception as e:
        print(f"\n--- UNKNOWN ERROR on chunk {i} ---: {e}")

    # pause between requests
    time.sleep(0.1)

print("\nAll chunks downloaded. Concatenating files ...")

# Concatenate all saved CSVs
all_files = [os.path.join(output_dir, path) for path in os.listdir(output_dir) if path.endswith(".csv")]
df_list = [pd.read_csv(path) for path in all_files]

if df_list:
    df_final = pd.concat(df_list, ignore_index=True)

    df_final.to_csv("bres_employment_5digsic_LAD.csv", index=False)
    print(f"Success! Final data saved to 'bres_employment_5digsic_LAD.csv' with {len(df_final)} rows.")
else:
    print("No dataframes to concatenate. Please check for errors in the download process.")


Starting download ... Will make 35 separate API calls.


Chunk 0 already downloaded. Skipping...
Chunk 1 already downloaded. Skipping...
Chunk 2 already downloaded. Skipping...
Chunk 3 already downloaded. Skipping...
Chunk 4 already downloaded. Skipping...
Chunk 5 already downloaded. Skipping...
Chunk 6 already downloaded. Skipping...
Chunk 7 already downloaded. Skipping...
Chunk 8 already downloaded. Skipping...
Chunk 9 already downloaded. Skipping...
Chunk 10 already downloaded. Skipping...
Chunk 11 already downloaded. Skipping...
Chunk 12 already downloaded. Skipping...
Chunk 13 already downloaded. Skipping...
Chunk 14 already downloaded. Skipping...
Chunk 15 already downloaded. Skipping...
Chunk 16 already downloaded. Skipping...
Chunk 17 already downloaded. Skipping...
Chunk 18 already downloaded. Skipping...
Chunk 19 already downloaded. Skipping...
Chunk 20 already downloaded. Skipping...
Chunk 21 already downloaded. Skipping...
Chunk 22 already downloaded. Skipping...
Chunk 23 already downloaded. Skipping...
Chunk 24 already downloade

Success! Final data saved to 'bres_employment_5digsic_LAD.csv' with 255150 rows.


In [169]:
# Read in concatenated dataframe

employment_SIC_LAD = pd.read_csv("bres_employment_5digsic_LAD.csv")

# Select columns of interest
employment_SIC_LAD = employment_SIC_LAD[["GEOGRAPHY_CODE", "GEOGRAPHY_NAME", "INDUSTRY_CODE", "OBS_VALUE"]]

employment_SIC_LAD.dtypes

GEOGRAPHY_CODE    object
GEOGRAPHY_NAME    object
INDUSTRY_CODE      int64
OBS_VALUE          int64
dtype: object

In [170]:
# Filter to IS-8 SIC codes only

IS8_LOOKUP = pd.read_csv("IS-8 SIC Lookup.csv")

IS8_LOOKUP.head()

# Full join first with employment data to introduce indicator of IS8 sector, later on we filter
# We need to do this on SIC digit level because some are defined at 2,3,4 etc

# Create different digit columns in the employment data before merging
employment_SIC_LAD["SIC_2digit"] = employment_SIC_LAD["INDUSTRY_CODE"].astype(str).str[:2]
employment_SIC_LAD["SIC_3digit"] = employment_SIC_LAD["INDUSTRY_CODE"].astype(str).str[:3]
employment_SIC_LAD["SIC_4digit"] = employment_SIC_LAD["INDUSTRY_CODE"].astype(str).str[:4]
employment_SIC_LAD["SIC_5digit"] = employment_SIC_LAD["INDUSTRY_CODE"].astype(str).str[:5]

IS8_LOOKUP['SIC'] = IS8_LOOKUP['SIC'].astype(str)
IS8_LOOKUP = IS8_LOOKUP[['SIC','Digit level','IS8 sector']]
IS8_LOOKUP = IS8_LOOKUP.rename(columns={'IS8 sector':'IS8_Sector'})

lookup_2dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 2].drop('Digit level',axis=1)
lookup_3dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 3].drop('Digit level',axis=1)
lookup_4dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 4].drop('Digit level',axis=1)
lookup_5dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 5].drop('Digit level',axis=1)


# 1. Merge 5-digit SICs

df_merged = pd.merge(employment_SIC_LAD, lookup_5dig, how='left', left_on='SIC_5digit', right_on='SIC', suffixes=('','_5'))

# 2. Merge 4-digit SICs

df_merged = pd.merge(df_merged, lookup_4dig, how='left', left_on='SIC_4digit', right_on='SIC', suffixes=('','_4'))

# 3. Merge 3-digit SICs

df_merged = pd.merge(df_merged, lookup_3dig, how='left', left_on='SIC_3digit', right_on='SIC', suffixes = ('','_3'))

# 4. Merge 2-digit SICs

df_merged = pd.merge(df_merged, lookup_2dig, how='left', left_on='SIC_2digit', right_on='SIC', suffixes=('','_2'))


# Combine all the digit level flags into one
# Rename the first flag column (from the 5-digit merge) for clarity
df_merged = df_merged.rename(columns={'IS8_Sector': 'IS8_Sector_5'})

# Coalesce the flags, starting with the most specific
df_merged['IS8_Final_Flag'] = (
    df_merged['IS8_Sector_5']
    .fillna(df_merged['IS8_Sector_4'])
    .fillna(df_merged['IS8_Sector_3'])
    .fillna(df_merged['IS8_Sector_2'])
)

# Drop all the intermediary flags
cols_to_drop = [
    'SIC', 'IS8_Sector_5', 
    'SIC_4', 'IS8_Sector_4', 
    'SIC_3', 'IS8_Sector_3', 
    'SIC_2', 'IS8_Sector_2',
]

df_final = df_merged.drop(columns=cols_to_drop, errors='ignore')

df_final.head()

,GEOGRAPHY_CODE,GEOGRAPHY_NAME,INDUSTRY_CODE,OBS_VALUE,SIC_2digit,SIC_3digit,SIC_4digit,SIC_5digit,IS8_Final_Flag
0,E06000001,Hartlepool,47781,0,47,477,4778,47781,NaN
1,E06000001,Hartlepool,47782,100,47,477,4778,47782,NaN
2,E06000001,Hartlepool,47789,75,47,477,4778,47789,NaN
3,E06000001,Hartlepool,47791,0,47,477,4779,47791,NaN
4,E06000001,Hartlepool,47799,40,47,477,4779,47799,NaN


In [171]:
# Keep only industries in the IS8

# Number of rows before 261800
row_count = df_final.shape[0]
print(f"Number of rows: {row_count}")

IS8_df = df_final[df_final['IS8_Final_Flag'].notnull()]

# Number of rows after 80850
row_count = IS8_df.shape[0]
print(f"Number of rows: {row_count}")

IS8_df.head(100)

Number of rows: 261800
Number of rows: 87500


,GEOGRAPHY_CODE,GEOGRAPHY_NAME,INDUSTRY_CODE,OBS_VALUE,SIC_2digit,SIC_3digit,SIC_4digit,SIC_5digit,IS8_Final_Flag
40960,E06000001,Hartlepool,58110,0,58,581,5811,58110,Creative Industries
40961,E06000001,Hartlepool,58110,0,58,581,5811,58110,Digital and Technology
40962,E06000001,Hartlepool,58120,0,58,581,5812,58120,Creative Industries
40963,E06000001,Hartlepool,58120,0,58,581,5812,58120,Digital and Technology
40964,E06000001,Hartlepool,58130,0,58,581,5813,58130,Creative Industries
...,...,...,...,...,...,...,...,...,...
41075,E06000003,Redcar and Cleveland,59140,0,59,591,5914,59140,Creative Industries
41086,E06000004,Stockton-on-Tees,58110,5,58,581,5811,58110,Creative Industries
41087,E06000004,Stockton-on-Tees,58110,5,58,581,5811,58110,Digital and Technology
41088,E06000004,Stockton-on-Tees,58120,0,58,581,5812,58120,Creative Industries


In [172]:
# Aggregate employment in IS8 industries to local authority

LAD_IS8_total = IS8_df.groupby(['GEOGRAPHY_CODE','GEOGRAPHY_NAME'], as_index=False).sum('OBS_VALUE')

LAD_IS8_total.head(100)

,GEOGRAPHY_CODE,GEOGRAPHY_NAME,INDUSTRY_CODE,OBS_VALUE
0,E06000001,Hartlepool,13067450,3820
1,E06000002,Middlesbrough,13067450,8755
2,E06000003,Redcar and Cleveland,13067450,4835
3,E06000004,Stockton-on-Tees,13067450,17650
4,E06000005,Darlington,13067450,11150
...,...,...,...,...
95,E07000072,Epping Forest,13067450,9860
96,E07000073,Harlow,13067450,8105
97,E07000074,Maldon,13067450,3895
98,E07000075,Rochford,13067450,4695


In [173]:
# Read in LAD 2023 SHAPEFILE
lad_shapefile_path = "Local_Authority_Districts_December_2023_Boundaries_UK_BGC_-6607102865052560878/LAD_DEC_2023_UK_BGC.shp"

lad_shp = gpd.read_file(lad_shapefile_path)
lad_shp

,LAD23CD,LAD23NM,LAD23NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,E06000001,Hartlepool,None,447160,531474,-1.27018,54.6761,338a54d0-e089-4a19-a10b-33a88632b2c9,"MULTIPOLYGON (((449812.866 525823.719, 449819...."
1,E06000002,Middlesbrough,None,451141,516887,-1.21099,54.5447,c46f7f23-b465-4f7e-900a-0def16eea429,"MULTIPOLYGON (((446860 517200.3, 446854.898 51..."
2,E06000003,Redcar and Cleveland,None,464361,519597,-1.00608,54.5675,bfb9468c-0951-46b7-996c-b8f076c634b2,"MULTIPOLYGON (((451747.397 520561.1, 451787.40..."
3,E06000004,Stockton-on-Tees,None,444940,518179,-1.30664,54.5569,1f58ac8f-80a5-4cdf-b62a-55b5154c0724,"MULTIPOLYGON (((446997.146 517642.744, 446963...."
4,E06000005,Darlington,None,428029,515648,-1.56835,54.5353,1ce00468-9b62-455e-bd4c-74b5ec0d672e,"POLYGON ((423475.701 524731.597, 423497.204 52..."
...,...,...,...,...,...,...,...,...,...
356,W06000020,Torfaen,Torfaen,327459,200480,-3.05101,51.6984,edc8f928-a9ad-41e0-9956-65173ef8ce22,"POLYGON ((323825.299 211337.105, 324481.004 21..."
357,W06000021,Monmouthshire,Sir Fynwy,337812,209231,-2.90280,51.7783,ae902fd3-dae3-4e5e-94db-f48ac9156c64,"MULTIPOLYGON (((345920.603 181082.095, 345897...."
358,W06000022,Newport,Casnewydd,337897,187432,-2.89769,51.5823,4fb8cc6b-cdfd-4661-8cdd-8b694b11b357,"MULTIPOLYGON (((334689.64 191800.97, 334695.39..."
359,W06000023,Powys,Powys,302329,273254,-3.43531,52.3486,00d90305-9508-477c-aa3a-61874a36237f,"MULTIPOLYGON (((270878.107 297590.695, 270715...."


In [174]:
# JOIN THE EMPLOYMENT DATA TO THE LAD SHAPEFILE
LAD_IS8_total = LAD_IS8_total.rename(columns={'GEOGRAPHY_CODE':'LAD23CD',
                                              'GEOGRAPHY_NAME':'LAD23NM'})


LAD_IS8_total_gdf = lad_shp.merge(
    LAD_IS8_total,
    on=['LAD23CD','LAD23NM'],
    how='inner'
)

if LAD_IS8_total_gdf.crs != "EPSG:4326":
    print("Converting CRS to EPSG:4326...")
    LAD_IS8_total_gdf = LAD_IS8_total_gdf.to_crs(epsg=4326)
    print("Conversion complete.")

alt.data_transformers.enable('json')

Converting CRS to EPSG:4326...
Conversion complete.


DataTransformerRegistry.enable('json')

In [175]:
# Plot MAP of IS8 employment in each LAD
# Maybe we want to do IS8 share of employment - would need some extra processing

chart = alt.Chart(LAD_IS8_total_gdf).mark_geoshape(
    stroke='black',  
    strokeWidth=0.2).encode(
        color=alt.Color('OBS_VALUE:Q', title=None,
                        scale=alt.Scale(scheme='blues')
        ),
        tooltip=[
            'LAD23CD:N', 
            'OBS_VALUE:Q'
        ]
).properties(
        title='IS8 employment', 
        width=700,  
        height=1000 
).project(
        type='transverseMercator'
    )

chart

alt.Chart(...)

In [ ]:
# FROM NOMIS API RETRIEVE TOTAL EMPLOYMENT FROM BRES FOR LADS
# This will be our denominator for shares

base_url = "https://www.nomisweb.co.uk/api/v01/dataset/NM_189_1.data.csv"

GEO_LIST_STR = "1778384897...1778384901,1778384941,1778384950,1778385143...1778385146,1778385159,1778384902...1778384905,1778384942,1778384943,1778384956,1778384957,1778385033...1778385044,1778385124...1778385138,1778384906...1778384910,1778384958,1778385139...1778385142,1778385154...1778385158,1778384911...1778384914,1778384954,1778384955,1778384965...1778384972,1778385045...1778385058,1778385066...1778385072,1778384915...1778384917,1778384944,1778385078...1778385085,1778385100...1778385104,1778385112...1778385117,1778385147...1778385153,1778384925...1778384928,1778384948,1778384949,1778384960...1778384964,1778384986...1778384997,1778385015...1778385020,1778385059...1778385065,1778385086...1778385088,1778385118...1778385123,1778385160...1778385192,1778384929...1778384940,1778384953,1778384981...1778384985,1778385004...1778385014,1778385021...1778385032,1778385073...1778385077,1778385089...1778385099,1778385105...1778385111,1778384918...1778384924,1778384945...1778384947,1778384951,1778384952,1778384973...1778384980,1778384998...1778385003,1778384959,1778385193...1778385246"

params = {
        'geography': GEO_LIST_STR,
        'date': 'latest',
        'industry':'37748736',
        'employment_status': '4',
        'measure':'1',
        'measures': '20100'
    #    'select': 'GEOGRAPHY_CODE, GEOGRAPHY_NAME, OBS_VALUE'
    }

response = requests.get(base_url, params=params)

        # Raise an error if the API returns 4xx or 5xx status code
response.raise_for_status()

        # Use io.StringIO to read the text content into a dataframe
chunk_df = pd.read_csv(io.StringIO(response.text))

        # Save the chunk to a file
chunk_df.to_csv('total_BRES_employment_LAD.csv', index=False)


In [178]:
# Read in TOTAL EMPLOYMENT, join to our df and create share column

total_employment_df = pd.read_csv('total_BRES_employment_LAD.csv')
total_employment_df = total_employment_df[['GEOGRAPHY_CODE','GEOGRAPHY_NAME','OBS_VALUE']]
total_employment_df = total_employment_df.rename(columns={'OBS_VALUE':'TOTAL_EMPLOYMENT',
                                                        'GEOGRAPHY_CODE':'LAD23CD',
                                                        'GEOGRAPHY_NAME':'LAD23NM'})



LAD_IS8_total_gdf = LAD_IS8_total_gdf.merge(total_employment_df, on=['LAD23CD','LAD23NM'], how='left')

#LAD_IS8_total_gdf['IS8_share_of_total_emp'] = LAD_IS8_total_gdf['OBS_VALUE'] / LAD_IS8_total_gdf['TOTAL_EMPLOYMENT'] * 100


In [179]:
# Create IS8 employment share of total employment column
LAD_IS8_total_gdf['IS8_share_of_total_emp'] = LAD_IS8_total_gdf['OBS_VALUE'] / LAD_IS8_total_gdf['TOTAL_EMPLOYMENT'] * 100
LAD_IS8_total_gdf

,LAD23CD,LAD23NM,LAD23NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry,INDUSTRY_CODE,OBS_VALUE,TOTAL_EMPLOYMENT,IS8_share_of_total_emp
0,E06000001,Hartlepool,None,447160,531474,-1.27018,54.6761,338a54d0-e089-4a19-a10b-33a88632b2c9,"MULTIPOLYGON (((-1.23001 54.62512, -1.22991 54...",13067450,3820,31000,12.322581
1,E06000002,Middlesbrough,None,451141,516887,-1.21099,54.5447,c46f7f23-b465-4f7e-900a-0def16eea429,"MULTIPOLYGON (((-1.27712 54.54791, -1.2772 54....",13067450,8755,65000,13.469231
2,E06000003,Redcar and Cleveland,None,464361,519597,-1.00608,54.5675,bfb9468c-0951-46b7-996c-b8f076c634b2,"MULTIPOLYGON (((-1.20098 54.57763, -1.20037 54...",13067450,4835,41000,11.792683
3,E06000004,Stockton-on-Tees,None,444940,518179,-1.30664,54.5569,1f58ac8f-80a5-4cdf-b62a-55b5154c0724,"MULTIPOLYGON (((-1.27493 54.55187, -1.27546 54...",13067450,17650,85000,20.764706
4,E06000005,Darlington,None,428029,515648,-1.56835,54.5353,1ce00468-9b62-455e-bd4c-74b5ec0d672e,"POLYGON ((-1.638 54.6172, -1.63767 54.6167, -1...",13067450,11150,56000,19.910714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,W06000020,Torfaen,Torfaen,327459,200480,-3.05101,51.6984,edc8f928-a9ad-41e0-9956-65173ef8ce22,"POLYGON ((-3.10597 51.79548, -3.09636 51.79155...",13067450,6530,37000,17.648649
346,W06000021,Monmouthshire,Sir Fynwy,337812,209231,-2.90280,51.7783,ae902fd3-dae3-4e5e-94db-f48ac9156c64,"MULTIPOLYGON (((-2.78092 51.52605, -2.78124 51...",13067450,6005,38000,15.802632
347,W06000022,Newport,Casnewydd,337897,187432,-2.89769,51.5823,4fb8cc6b-cdfd-4661-8cdd-8b694b11b357,"MULTIPOLYGON (((-2.94479 51.62123, -2.9447 51....",13067450,19180,78000,24.589744
348,W06000023,Powys,Powys,302329,273254,-3.43531,52.3486,00d90305-9508-477c-aa3a-61874a36237f,"MULTIPOLYGON (((-3.90623 52.5608, -3.90861 52....",13067450,7645,58000,13.181034


In [180]:
# Quick check 

# Highest shares - London apart from Dacorum (herts), Bracknell Forest (berks) and Rushmoor (berks)
LAD_IS8_total_gdf.nlargest(10, ['IS8_share_of_total_emp'])

# Lowest shares - Isles of Scilly, Pembrokeshire, Shetland Islands, Clackmannanshire, Dumfries and Galloway etc
#LAD_IS8_total_gdf.nsmallest(10, ['IS8_share_of_total_emp'])


,LAD23CD,LAD23NM,LAD23NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry,INDUSTRY_CODE,OBS_VALUE,TOTAL_EMPLOYMENT,IS8_share_of_total_emp
263,E09000001,City of London,None,532382,181358,-0.09351,51.5156,2f933b97-47bd-49bf-b90c-0a2db58de6d6,"MULTIPOLYGON (((-0.10415 51.50854, -0.10468 51...",13067450,576270,676000,85.247041
119,E07000096,Dacorum,None,500084,208745,-0.55098,51.7685,c62eb69d-2bab-4224-a320-b21e7b2bb89d,"POLYGON ((-0.46587 51.85081, -0.4641 51.85015,...",13067450,82255,134000,61.384328
292,E09000030,Tower Hamlets,None,536340,181452,-0.03647,51.5155,244af541-6503-4dc4-93e5-18c0d12eff7c,"POLYGON ((-0.02955 51.54309, -0.02899 51.54227...",13067450,211660,350000,60.474286
281,E09000019,Islington,None,531160,184645,-0.10989,51.5455,aaede633-8c06-4104-84fe-f5ef40e5a9e4,"POLYGON ((-0.11755 51.57414, -0.11555 51.57258...",13067450,160895,271000,59.370849
269,E09000007,Camden,None,527491,184283,-0.16291,51.5430,d407a7c6-c501-4d81-8aff-f028b64206fc,"POLYGON ((-0.16729 51.57296, -0.16454 51.5725,...",13067450,229540,446000,51.466368
290,E09000028,Southwark,None,533945,175869,-0.07308,51.4659,62ad3906-708d-4749-8253-08784c8bdce3,"POLYGON ((-0.10468 51.50861, -0.10468 51.50841...",13067450,146900,297000,49.461279
275,E09000013,Hammersmith and Fulham,None,523867,177993,-0.21735,51.4873,5b137210-44b6-4409-93cf-a8f76b866628,"POLYGON ((-0.23382 51.53233, -0.23072 51.53087...",13067450,72365,147000,49.227891
295,E09000033,Westminster,None,528268,180871,-0.15295,51.5122,e9bc0792-dafe-43b9-b3aa-cba9b477861a,"POLYGON ((-0.17413 51.53826, -0.17348 51.53765...",13067450,406165,837000,48.526284
33,E06000036,Bracknell Forest,None,488168,168791,-0.73364,51.4113,5f6d4551-ea66-46e3-9f4c-785b257cb47a,"POLYGON ((-0.76075 51.45911, -0.75733 51.45902...",13067450,31935,67000,47.664179
274,E09000012,Hackney,None,534560,185787,-0.06045,51.5549,97725b6c-dbb4-4b24-8232-6c34a3532cf2,"POLYGON ((-0.06116 51.57779, -0.05843 51.57256...",13067450,82330,173000,47.589595


In [181]:
# Plot MAP of IS8 employment share out of total employment

chart = alt.Chart(LAD_IS8_total_gdf).mark_geoshape(
    stroke='black',  
    strokeWidth=0.2).encode(
        color=alt.Color('IS8_share_of_total_emp:Q', title=None,
                        scale=alt.Scale(scheme='blues')
        ),
        tooltip=[
            'LAD23CD:N', 
            'IS8_share_of_total_emp:Q'
        ]
).properties(
        title='Share of total employment in IS8 sectors', 
        width=700,  
        height=1000 
).project(
        type='transverseMercator'
    )

chart

alt.Chart(...)

In [ ]:
# BREAKDOWN BY EACH SECTOR WITH SIDE BY SIDE MAP PLOTS

# First, aggregate employment by each IS8 sector rather than as a total
LAD_IS8_sectoral_emp = IS8_df.groupby(['GEOGRAPHY_CODE','GEOGRAPHY_NAME','IS8_Final_Flag'], as_index=False).sum('OBS_VALUE')
LAD_IS8_sectoral_emp = LAD_IS8_sectoral_emp.rename(columns={'GEOGRAPHY_CODE':'LAD23CD',
                                                            'GEOGRAPHY_NAME':'LAD23NM'})

LAD_IS8_sectoral_emp.columns


Index(['LAD23CD', 'LAD23NM', 'IS8_Final_Flag', 'INDUSTRY_CODE', 'OBS_VALUE'], dtype='object')

In [204]:
# Join LAD total employment so we can calculate shares again
LAD_IS8_sectoral_emp = LAD_IS8_sectoral_emp.merge(total_employment_df, on=['LAD23CD','LAD23NM'], how='left')

LAD_IS8_sectoral_emp.dtypes

LAD23CD             object
LAD23NM             object
IS8_Final_Flag      object
INDUSTRY_CODE        int64
OBS_VALUE            int64
TOTAL_EMPLOYMENT     int64
dtype: object

In [206]:
# Calculate the share
LAD_IS8_sectoral_emp['sector_share_of_total_emp'] = LAD_IS8_sectoral_emp['OBS_VALUE'] / LAD_IS8_sectoral_emp['TOTAL_EMPLOYMENT']

LAD_IS8_sectoral_emp.head(100)

,LAD23CD,LAD23NM,IS8_Final_Flag,INDUSTRY_CODE,OBS_VALUE,TOTAL_EMPLOYMENT,sector_share_of_total_emp
0,E06000001,Hartlepool,Advanced manufacturing,1770920,1060,31000,0.034194
1,E06000001,Hartlepool,Creative Industries,3450921,740,31000,0.023871
2,E06000001,Hartlepool,Defence sector,55800,0,31000,0.000000
3,E06000001,Hartlepool,Digital and Technology,2181722,775,31000,0.025000
4,E06000001,Hartlepool,Financial Services,2207575,110,31000,0.003548
...,...,...,...,...,...,...,...
95,E06000014,York,Financial Services,2207575,4630,121000,0.038264
96,E06000014,York,Life Sciences,175610,335,121000,0.002769
97,E06000014,York,Professional and Business Services,3224902,10585,121000,0.087479
98,E06000015,Derby,Advanced manufacturing,1770920,10980,140000,0.078429


In [ ]:
# Pivot to wide so we have columns for industries
LAD_IS8_sectoral_emp_wide = LAD_IS8_sectoral_emp.pivot(
    index=['LAD23CD','LAD23NM'],
    columns='IS8_Final_Flag',
    values='sector_share_of_total_emp'
)

LAD_IS8_sectoral_emp_wide = LAD_IS8_sectoral_emp_wide.reset_index().rename_axis(columns=None)


In [208]:
# Join with the LAD shapefile

LAD_IS8_sectoral_emp_wide_gdf = lad_shp.merge(
    LAD_IS8_sectoral_emp_wide,
    on=['LAD23CD','LAD23NM'],
    how='inner'
)

if LAD_IS8_sectoral_emp_wide_gdf.crs != "EPSG:4326":
    print("Converting CRS to EPSG:4326...")
    LAD_IS8_sectoral_emp_wide_gdf = LAD_IS8_sectoral_emp_wide_gdf.to_crs(epsg=4326)
    print("Conversion complete.")

alt.data_transformers.enable('json')
LAD_IS8_sectoral_emp_wide_gdf

Converting CRS to EPSG:4326...
Conversion complete.


,LAD23CD,LAD23NM,LAD23NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry,Advanced manufacturing,Creative Industries,Defence sector,Digital and Technology,Financial Services,Life Sciences,Professional and Business Services
0,E06000001,Hartlepool,None,447160,531474,-1.27018,54.6761,338a54d0-e089-4a19-a10b-33a88632b2c9,"MULTIPOLYGON (((-1.23001 54.62512, -1.22991 54...",0.034194,0.023871,0.000000,0.025000,0.003548,0.000323,0.036290
1,E06000002,Middlesbrough,None,451141,516887,-1.21099,54.5447,c46f7f23-b465-4f7e-900a-0def16eea429,"MULTIPOLYGON (((-1.27712 54.54791, -1.2772 54....",0.009308,0.022308,0.000000,0.017077,0.015538,0.000846,0.069615
2,E06000003,Redcar and Cleveland,None,464361,519597,-1.00608,54.5675,bfb9468c-0951-46b7-996c-b8f076c634b2,"MULTIPOLYGON (((-1.20098 54.57763, -1.20037 54...",0.016707,0.007683,0.000000,0.038049,0.004146,0.002927,0.048415
3,E06000004,Stockton-on-Tees,None,444940,518179,-1.30664,54.5569,1f58ac8f-80a5-4cdf-b62a-55b5154c0724,"MULTIPOLYGON (((-1.27493 54.55187, -1.27546 54...",0.035588,0.020706,0.000000,0.067824,0.011824,0.001000,0.070706
4,E06000005,Darlington,None,428029,515648,-1.56835,54.5353,1ce00468-9b62-455e-bd4c-74b5ec0d672e,"POLYGON ((-1.638 54.6172, -1.63767 54.6167, -1...",0.027589,0.017857,0.000000,0.022768,0.027589,0.000089,0.103214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,W06000020,Torfaen,Torfaen,327459,200480,-3.05101,51.6984,edc8f928-a9ad-41e0-9956-65173ef8ce22,"POLYGON ((-3.10597 51.79548, -3.09636 51.79155...",0.067027,0.018243,0.000000,0.033784,0.011081,0.000000,0.046351
346,W06000021,Monmouthshire,Sir Fynwy,337812,209231,-2.90280,51.7783,ae902fd3-dae3-4e5e-94db-f48ac9156c64,"MULTIPOLYGON (((-2.78092 51.52605, -2.78124 51...",0.013158,0.025526,0.015789,0.030658,0.008684,0.003553,0.060658
347,W06000022,Newport,Casnewydd,337897,187432,-2.89769,51.5823,4fb8cc6b-cdfd-4661-8cdd-8b694b11b357,"MULTIPOLYGON (((-2.94479 51.62123, -2.9447 51....",0.022372,0.030192,0.000000,0.035641,0.052244,0.000897,0.104551
348,W06000023,Powys,Powys,302329,273254,-3.43531,52.3486,00d90305-9508-477c-aa3a-61874a36237f,"MULTIPOLYGON (((-3.90623 52.5608, -3.90861 52....",0.007500,0.021983,0.000345,0.038190,0.006810,0.006897,0.050086


In [187]:
# PLOT ALL SECTORS ON A MAP SIDE BY SIDE
sectors = ['Advanced manufacturing','Creative Industries','Defence sector','Digital and Technology','Financial Services','Life Sciences','Professional and Business Services']

chart_list = []

for sector_name in sectors:
    chart = alt.Chart(LAD_IS8_sectoral_emp_wide_gdf).mark_geoshape(
    stroke='black',  
    strokeWidth=0.2).encode(
        color=alt.Color(f'{sector_name}:Q', title=None,
                        scale=alt.Scale(scheme='blues')
        ),
        tooltip=[
            'LAD23CD:N', 
            f'{sector_name}:Q'
        ]
).properties(
        title=sector_name, 
        width=700,  
        height=1000 
).project(
        type='transverseMercator'
    )

    chart_list.append(chart)

if len(chart_list) == 7:
    final_grid = alt.vconcat(
        alt.hconcat(chart_list[0], chart_list[1], chart_list[2], chart_list[3]).resolve_scale(
        color='independent'
    ).resolve_legend(
        color='independent'
    ),
        alt.hconcat(chart_list[4], chart_list[5], chart_list[6]).resolve_scale(
        color='independent'
    ).resolve_legend(
        color='independent'
    )
    )

final_grid

alt.VConcatChart(...)

In [188]:
# DEFINE CLUSTER LOCATIONS AS DEFINED IN THE GOV INDUSTRIAL STRATEGY
#https://assets.publishing.service.gov.uk/media/68585f10c9b3bb1663ab9072/industrial_strategy_technical_annex.pdf#page=24.15

# These are loosely defined in the document with very rough dots on the map
# I asked Gemini to assign each 'cluster' such as East Midlands CA a local authority code ()

cluster_data_list = [
    {'sector':'Advanced manufacturing', 'LAD23CD':'E06000015', 'cluster_name':'East Midlands'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E08000024', 'cluster_name':'North East'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E08000003', 'cluster_name':'North West (Greater Manchester CA)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E06000006', 'cluster_name':'North West (Liverpool City Region)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E07000126', 'cluster_name':'North West (Cheshire, Lancashire)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E07000008', 'cluster_name':'Oxford to Cambridge Growth Corridor'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'S12000036', 'cluster_name':'Scotland (Edinburgh)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E06000025', 'cluster_name':'South West (West of England CA, Somerset, Gloucestershire)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'W06000005', 'cluster_name':'Wales (Wrexham and Flintshire)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E08000026', 'cluster_name':'West Midlands (West Midlands CA, Warwickshire)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E08000018', 'cluster_name':'Yorkshire (South Yorkshire CA)'},
    {'sector':'Advanced manufacturing', 'LAD23CD':'E06000014', 'cluster_name':'Yorkshire (York and North Yorkshire CA)'},
    {'sector':'Creative Industries', 'LAD23CD':'W06000015', 'cluster_name':'Cardiff'},
    {'sector':'Creative Industries', 'LAD23CD':'S12000042', 'cluster_name':'Dundee'},
    {'sector':'Creative Industries', 'LAD23CD':'S12000036', 'cluster_name':'Edinburgh'},
    {'sector':'Creative Industries', 'LAD23CD':'S12000049', 'cluster_name':'Glasgow'},
    {'sector':'Creative Industries', 'LAD23CD':'E09000033', 'cluster_name':' Greater London Authority'},
    {'sector':'Creative Industries', 'LAD23CD':'E08000006', 'cluster_name':' Greater Manchester CA'},
    {'sector':'Creative Industries', 'LAD23CD':'E08000012', 'cluster_name':' Liverpool City Region'},
    {'sector':'Creative Industries', 'LAD23CD':'E06000002', 'cluster_name':'North East CA'},
    {'sector':'Creative Industries', 'LAD23CD':'E08000025', 'cluster_name':'West Midlands CA'},
    {'sector':'Creative Industries', 'LAD23CD':'E06000023', 'cluster_name':'West of England CA'},
    {'sector':'Creative Industries', 'LAD23CD':'E08000035', 'cluster_name':'West Yorkshire CA'},
    {'sector':'Digital and Technology', 'LAD23CD':'E09000012', 'cluster_name':'Greater London Authority'},
    {'sector':'Digital and Technology', 'LAD23CD':'E08000003', 'cluster_name':'Greater Manchester CA'},
    {'sector':'Digital and Technology', 'LAD23CD':'E08000012', 'cluster_name':'Liverpool City Region'},
    {'sector':'Digital and Technology', 'LAD23CD':'E07000178', 'cluster_name':'Oxford to Cambridge Growth Corridor (Oxford)'},
    {'sector':'Digital and Technology', 'LAD23CD':'E07000008', 'cluster_name':'Oxford to Cambridge Growth Corridor (Cambridge)'},
    {'sector':'Digital and Technology', 'LAD23CD':'S12000036', 'cluster_name':'Scotland (Dundee, Edinburgh, Glasgow)'},
    {'sector':'Digital and Technology', 'LAD23CD':'W06000015', 'cluster_name':'Wales (Cardiff, Swansea, Newport)'},
    {'sector':'Digital and Technology', 'LAD23CD':'E08000025', 'cluster_name':'West Midlands CA'},
    {'sector':'Digital and Technology', 'LAD23CD':'E06000023', 'cluster_name':'West of England CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E07000130', 'cluster_name':'East Midlands CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E09000013', 'cluster_name':'North East CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E06000003', 'cluster_name':'Tees Valley CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E08000003', 'cluster_name':'Greater Manchester CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E08000012', 'cluster_name':'Liverpool City Region'},
    {'sector':'Life Sciences', 'LAD23CD':'E06000049', 'cluster_name':'Cheshire'},
    {'sector':'Life Sciences', 'LAD23CD':'E07000180', 'cluster_name':'Oxford to Cambridge Growth Corridor'},
    {'sector':'Life Sciences', 'LAD23CD':'E07000012', 'cluster_name':'Oxford to Cambridge Growth Corridor'},
    {'sector':'Life Sciences', 'LAD23CD':'S12000036', 'cluster_name':'Edinburgh'},
    {'sector':'Life Sciences', 'LAD23CD':'S12000049', 'cluster_name':'Glasgow'},
    {'sector':'Life Sciences', 'LAD23CD':'S12000042', 'cluster_name':'Dundee'},
    {'sector':'Life Sciences', 'LAD23CD':'S12000033', 'cluster_name':'Aberdeen'},
    {'sector':'Life Sciences', 'LAD23CD':'W06000015', 'cluster_name':'Cardiff'},
    {'sector':'Life Sciences', 'LAD23CD':'W06000011', 'cluster_name':'Swansea'},
    {'sector':'Life Sciences', 'LAD23CD':'W06000006', 'cluster_name':'Wrexham'},
    {'sector':'Life Sciences', 'LAD23CD':'E08000025', 'cluster_name':'West Midlands CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E08000019', 'cluster_name':'South Yorkshire CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E06000014', 'cluster_name':'York and North Yorkshire CA'},
    {'sector':'Life Sciences', 'LAD23CD':'E06000010', 'cluster_name':'Humber'},
]

cluster_df = pd.DataFrame(cluster_data_list)


In [189]:
# Merge df with LAD shapefile for geocoordinates
cluster_gdf = pd.merge(
    cluster_df,
    lad_shp,
    on='LAD23CD',
    how='inner'
)

cluster_gdf = cluster_gdf[
    ['sector', 'cluster_name', 'LAD23CD', 'LAT', 'LONG']
].copy()

In [ ]:

# FINAL FOR ARTICLE - Chart four sectors with clusters identified in the IS layered on top

sectors = ['Advanced manufacturing','Creative Industries','Digital and Technology','Life Sciences']

color_schemes = ['blues', 'tealblues', 'teals', 'greens']

chart_list = []

for i, sector_name in enumerate(sectors):
    
    base_map = alt.Chart(LAD_IS8_sectoral_emp_wide_gdf).mark_geoshape(
        stroke='black',  
        strokeWidth=0.2
    ).encode(
        color=alt.Color(f'{sector_name}:Q', title='Share of total employment',
                        scale=alt.Scale(scheme='yellowgreenblue'),
                        legend=alt.Legend(format='%')
        ),
        tooltip=[
            'LAD23NM:N', 
            alt.Tooltip(f'{sector_name}:Q', 
                      title='Share of total employment', 
                      format='%')]
    ).properties(
        title=alt.TitleParams(text=sector_name, fontSize=20, fontWeight="normal"),
        width=400,  
        height=700 
    ).project(
        type='transverseMercator' 
    )

    # Create points fresh for each sector (don't reuse a base chart)
    sector_points = alt.Chart(cluster_gdf).mark_circle(
        color="black",
        size=80,
        opacity=0.8,
    ).encode(
        latitude='LAT:Q',
        longitude='LONG:Q',
        tooltip=['cluster_name:N','LAD23CD:N','sector:N']
    ).transform_filter(
        alt.datum.sector == sector_name
    ).project(
        type='transverseMercator'
    )

    # Layer the base map and the sector-specific points
    combined_chart = base_map + sector_points
    
    chart_list.append(combined_chart)

if len(chart_list) == 4:
    final_grid = alt.vconcat(
        alt.hconcat(chart_list[0], chart_list[1]).resolve_scale(
        color='independent'
    ).resolve_legend(
        color='independent'
    ),
    alt.hconcat(chart_list[2], chart_list[3]).resolve_scale(
        color='independent'
    ).resolve_legend(
        color='independent'
    )
    )

final_grid

final_grid.save('IS_sector_employment_LAD_2.png', scale_factor=2)
final_grid.save('IS_sector_employment_LAD_2.json')
